# SEL635/SEL5724 - Visão Computacional Aplicada a Robótica



**Aluno:** Luiza Rodrigues Cardoso

**NUSP:** 14593332

Versão do documento: 1.2

# Lista 2 - Transformação de Coordenadas


## Preparação do ambiente e definição de funções de exercícios anteriores

In [1]:
# Importar as bibliotecas que serão utilizadas
import numpy as np
import matplotlib.pyplot as plt
import math


Defina a seguir as funções implementadas em exercícios anteriores que serão utilizadas nessa Tarefa:

In [14]:
def proj_perspectiva(Pc,f):
  (M,N) = np.shape(Pc) # recupera o tamanho da matriz (N linhas, M colunas)
  if(M!=3):
    print('Aviso: a matriz Pc usada como entrada não possui três linhas!')

  pc = np.zeros((3,N))  # Inicializa o vetor pc de dimensão 3xN com zeros
  
  # Encontra a projeção perspectiva dos pontos no plano da imagem em [mm]:
  ######
  # Coloque aqui o seu código para calcular a matriz pc
  #Obtendo as coordenadas do sistema em relação ao mundo
  Xc = Pc[0,:]
  Yc = Pc[1,:]
  Zc = Pc[2,:]

 

  #aplicando o fator de correção q
  x = f*Xc/Zc
  y = f*Yc/Zc
      
  #matriz no sistema de coordenas da camera
  pc[0,:] = x
  pc[1,:] = y
  pc[2,:] = 1 #coordenada homogenea

  ######

  return pc


def camera_to_image(pc,sx,sy,ox,oy):
  (M,N) = np.shape(pc) # recupera o tamanho da matriz (N linhas, M colunas)
  if(M!=3):
    print('Aviso: a matriz pc usada como entrada não possui três linhas!')

  p_im = np.zeros((2,N))  # Inicializa o vetor p de dimensão 2xN com zeros

  # Converte as coordenadas dos pontos pc do sistema da câmera em [mm] para o sistema de coordenadas da imagem em [px]:
  #obtendo as coordenadas em relação ao sistema da camera
  x_im = pc[0,:]
  y_im = pc[1,:]
  
  #aplicando uma correção em relação à origem dos sistemas e passando da medida real(mm) para medidas discretas(pixel)
  x = ox - (x_im/sx)    
  y = oy - (y_im/sy)
      
  #matriz de coordenadas no sistema da imagem(x,y) em pixels
  p_im[0,:] = x
  p_im[1,:] = y
    
  
  ######

  return p_im


## Padrão de Calibração

A função `Pw = calibration_pattern(n,d)` definida a seguir retorna uma matriz de pontos `Pw` de dimensão $3 \times (3n+7)$, em que cada coluna da matriz representa a coordenada em $[mm]$ de um ponto do padrão de calibração em um sistema de coordenadas local fixo neste padrão de calibração $\{P\}$. O espaçamento entre os pontos é dado por $d$ em $[mm]$. O padrão de calibração é composto por $3n+7$ pontos distribuídos em 3 planos mutualmente ortogonais, na seguinte sequencia:

* Os 7 primeiros pontos são os pontos extremos das três faces ortogonais e
possuem as coordenadas na sequencia: $[0,0,0]^T$, $[n*d,0,0]^T$, $[0,n*d,0]^T$, $[0,0,n*d]^T$, $[n*d,n*d,0]^T$, $[n*d,0, n*d]^T$, $[0,n*d,n*d]^T$;
* Os demais pontos seguem uma sequencia começando do ponto $[0,0,0]^T$, sendo que $P$ terá repetição de pontos ao longo dos eixos $x$, $y$ e $z$, onde os planos se intersectam.





In [3]:
def calibration_pattern(n,d):
    Pw = np.zeros((3,3*n*n+7))
    Pw[:,0] = [0, 0, 0]
    Pw[:,1] = [n*d, 0, 0]
    Pw[:,2] = [0, n*d, 0]
    Pw[:,3]=[0, 0, n*d];
    Pw[:,4]=[n*d, n*d, 0];
    Pw[:,5]=[n*d, 0, n*d];
    Pw[:,6]=[0, n*d, n*d];

    for i in range(n):
        for j in range(n):
            Pw[:,n*i+j+7]= [d*j, d*i, 0]
            Pw[:,n*n + n*i+j+7]= [0, d*j, d*i];
            Pw[:,2*n*n + n*i+j+7]= [d*i, 0, d*j];

    return Pw

## Imagens do Padrão de Calibração

Considere que o sistema de coordenadas do mundo coincide com o sistema de coordenadas fixo no padrão de calibração, ou seja, $\{W\}=\{P\}$.

Considere que duas câmeras idênticas foram colocadas no ambiente para observar um padrão de calibração com $n=10$ e $d=100$.

As câmeras possuem os seguintes parâmetros intrínsecos:  distância focal $f=12mm$, $s_x=s_y=0.01$, $o_x=800$, $o_y=600$, para $s_x$ e $s_y$ dados em [mm/pixel], $o_x$ e $o_y$ dados em [pixel].

Considere também que as câmeras fornecem imagens de tamanho $1600 \times 1200$ pixels.


As câmeras foram posicionadas no ambiente da seguinte forma:



* **Camera 1**:  Considere que inicialmente o sistema de coordenada da primeira câmera $\{C_1\}$ coincide com o sistema de coordenada do mundo $\{W\}$. O sistema de coordenadas da câmera é primeiramente rotacionado de $20$ graus em relação ao seu eixo $z$, depois rotacionado de $-140$ graus em relação ao eixo $y$ do seu sistema atual, e então rotacionado de $25$ graus em relação a $x$ do seu sistema atual. Finalmente, a origem da câmera é posicionada na coordenada $[1600,1800,1700]^T$ dada em [mm] em relação ao sistema de coordenadas do mundo.

* **Camera 2**: Considere que inicialmente o sistema de coordenada da segunda câmera $\{C_2\}$ coincide com o sistema de coordenada do mundo $\{W\}$. Então, o sistema de coordenadas da câmera é rotacionado de $5$ graus em relação ao seu eixo $x$, depois rotacionado de $-120$ graus em relação ao eixo $y$ do sistema de coordenadas do mundo, e então rotacionado de $35$ graus em relação ao eixo $z$ do sistema de coordenadas do mundo. Finalmente, a origem da câmera é colocada na coordenada $[1700,1500,1600]^T$ dada em [mm] em relação ao sistema de coordenadas do mundo.



### Exercícios:

1. Qual a sequencia de rotações utilizada para obter a rotação que representa a orientação de cada câmera em relação ao mundo, ou seja, $^wR_{c1}$ e $^wR_{c2}$? Quais os valores de $^wR_{c1}$ e $^wR_{c2}$?

<responda aqui>





In [ ]:
def Rx(theta):
    theta = np.radians(theta)
    R = np.array([[1, 0, 0], [0, np.cos(theta), -np.sin(theta)], [0, np.sin(theta), np.cos(theta)]])
    return R
def Ry(theta):
    theta = np.radians(theta)
    R = np.array([[np.cos(theta), 0, np.sin(theta)], [0, 1, 0], [-np.sin(theta), 0, np.cos(theta)]])
    return R
def Rz(theta):
    theta = np.radians(theta)
    R = np.array([[np.cos(theta), -np.sin(theta), 0], [np.sin(theta), np.cos(theta), 0], [0, 0, 1]])
    return R
def Homogenea(R,t):
    t = t.reshape(-1, 1) #vetor de translação (formato de vetor coluna)
    H = np.concatenate((R,t),axis=1) #concatena na horizontal a matriz de rotação e vetor de translação
    temp_v = np.array([0,0,0,1]).reshape(1, -1) #ultima linha da matriz homogênia (formato de vetor linha)
    H = np.concatenate((H,temp_v),axis=0) #concatena na vertical a ultima linha da matriz de homografia
    return H

In [ ]:
## Insira seu codigo aqui para calcular e mostrar a matriz de rotação da câmera 1 em relação ao sistema de coordenadas do mundo
#todas rotação são em relação ao sistema de coordenadas da camera/eixos atuais
r11 = Rz(20) #rotação de 20° em relação ao eixo z
r12 = Ry(-140) #rotação de -140° em relação ao eixo y
r13 = Rx(25) #rotação de 25° em relação ao eixo x

#matriz geral de rotação
Rwc1 = r11@r12@r13 
print('Rwc1 =\n',Rwc1)


## Insira seu codigo aqui para calcular e mostrar a matriz de rotação da câmera 2 em relação ao sistema de coordenadas do mundo
#rotações em relação ao eixo da camera e do mundo/ rotações mistas
r21 = Rx(5) #rotação em relação ao eixo x, em relação a camera
r22 = Ry(-120) #rotação em relação ao eixo y, em relação ao mundo
r23 = Rz(35) #rotação em relação ao eixo z, em relação ao mundo

#No enunciado diz que as rotações em Z e y ocorrem em relação ao mundo e a rotação em X em relação a camera
#Assim, foi necessário aplicar as rotação relacionadas ao mundo antes dos eixos atuais
Rwc2 = r23 @ r22 @r21 
print('Rwc2 =\n',Rwc2)



-2.443460952792061
Rwc1 =
 [[-0.71984631 -0.56524657 -0.40288658]
 [-0.26200263  0.75873967 -0.59637969]
 [ 0.64278761 -0.32374437 -0.69427204]]
-2.0943951023931953
Rwc2 =
 [[-0.40957602 -0.63322265 -0.65671649]
 [-0.28678822  0.7727419  -0.56623536]
 [ 0.8660254  -0.04357787 -0.49809735]]


Resultado esperado:
```
Rwc1 =
[[-0.71984631 -0.56524657 -0.40288658]
 [-0.26200263  0.75873967 -0.59637969]
 [ 0.64278761 -0.32374437 -0.69427204]]
Rwc2 =
[[-0.40957602 -0.63322265 -0.65671649]
 [-0.28678822  0.7727419  -0.56623536]
 [ 0.8660254  -0.04357787 -0.49809735]]
 ```

2. Quais os valores da posição de cada câmera em relação ao mundo, ou seja, $^wT_{c1}$ e $^wT_{c2}$?

In [ ]:
## Insira seu codigo aqui para calcular e mostrar a posição da câmera 1 em relação ao sistema de coordenadas do mundo
#Twc1 =


## Insira seu codigo aqui para calcular e mostrar a posição da câmera 2 em relação ao sistema de coordenadas do mundo
#Twc2 =






3. Implemente uma funcao `Pc = world_to_camera(Pw,Rwc,Twc)` que converte as coordenadas de pontos descritas no sistema de coordenadas do mundo, ${^wP}$, para coordenadas no sistema de coordenadas da câmera, ${^cP}$. As entradas desta função são: uma matriz `Pw` de dimensão $3 \times N$ cujas colunas representam as coordenadas 3D de $N$ pontos descritas no sistema $\{W\}$; a matriz de rotação `Rwc` que descreve a orientação do sistema $\{C\}$ em relação ao sistema da câmera $\{W\}$, o vetor de translação `Twc` que descreve a origem do sistema $\{C\}$ em relação ao sistema $\{W\}$.


In [ ]:
def world_to_camera(Pw,Rwc,Twc):
    (M,N) = np.shape(Pw) # recupera o tamanho da matriz (M linhas, N colunas)

    # Coloque seu codigo aqui




    return Pc



4. Implemente a função `p_im = world_to_image(Pw,Rwc,Twc,f,sx,sy,ox,oy)` que retorna as coordenadas em pixel na imagem da câmera  correpondentes a projeção perspectiva dos pontos ${^wP}$ descritos no sistema de coordenadas do mundo. Observação: Utilize as funções já implementadas `world_to_camera`, `perspective`, e `camera_to_image`.


In [ ]:
def world_to_image(Pw,Rwc,Twc,f,sx,sy,ox,oy):

    # Coloque seu codigo aqui




    return p_im


6. Teste as funções implementadas para gerar duas figuras que representam as imagens de um padrão de calibração, construído com `n=10` e `d=100`, obtidas pelas câmeras 1 e 2 definidas no início deste documento. Utilize a função `plot_image(p_im, size_x, size_y)` implementada a seguir para mostrar uma figura que representa a imagem dos pontos `p_im` fornecidos em pixel. Além de `p_im`, a função também possui como entrada o tamanho horizontal da imagem `size_x` e o tamanho vertical da imagem `size_y` em pixel. Observe que o gráfico possui origem no canto superior esquerdo, eixo x positivo ao longo da horizontal da esquerda para direita, e eixo y positivo ao longo da vertical do topo para baixo. (veja ao final deste documento exemplo da imagem que deve ser produzida)

In [5]:
def plot_image(p_im,size_x,size_y):
    plt.figure()
    plt.plot(p_im[0,4:],p_im[1,4:],'bo',mfc='none')
    plt.plot(p_im[0,0],p_im[1,0],'ko',mfc='none')
    plt.plot(p_im[0,1],p_im[1,1],'co',mfc='none')
    plt.plot(p_im[0,2],p_im[1,2],'ro',mfc='none')
    plt.plot(p_im[0,3],p_im[1,3],'go',mfc='none')
    plt.axis([0, size_x, size_y, 0])

In [ ]:
# Código para gerar as imagens obtidas pelas câmeras

Pw = calibration_pattern(n=10,d=100) # pontos do padrão de calibração

# Coloque seu código aqui para gerar a imagem obtida pela câmera 1
#p_im1 =
plot_image(p_im1,size_x=1600,size_y=1200)
print('p_im1 = ',p_im1[:,0:6])

# Coloque seu código aqui para gerar a imagem obtida pela câmera 2
#p_im2 =
plot_image(p_im2,size_x=1600,size_y=1200)
print('p_im2 = ',p_im2[:,0:6])


### Imagens esperadas do Padrão de Calibração

Posição na imagem dos 6 primeiros do padrão de calibração:

```
p_im1 =  [[ 580.30924478  890.99327337  659.97269703  161.147924   1085.12209782
   497.82829469]
 [ 563.13981745  828.99888456  158.06751211  727.78910845  421.4805493
  1132.95125045]]
p_im2 =  [[ 912.57533459 1181.05516852 1098.27603088  478.43941228 1544.68802215
   652.77539125]
 [ 605.60739093  968.16479952  184.88407634  629.93210835  501.32882709
  1114.73700712]
```

A Figura 1 e 2 mostram as imagens do padrão de calibração esperadas para a câmera 1 e 2 respectivamente.


<div>
<img src='https://drive.google.com/uc?id=15N_OAMHxABu1OYHaJ8puM-NosodEKoKw' width="500"/>
</div>
Figura 1 - Imagem do padrão de calibração obtida pela câmera 1


<div>
<img src='https://drive.google.com/uc?id=1N_bHV5sYUH2W3VXgDLSSOs0ILKOqcqz2' width="500"/>
</div>
Figura 2 - Imagem do padrão de calibração obtida pela câmera 2

